# Urban Traffic Palm Springs Convention Center
Simulate the urban traffic from the city of Palm Springs nearby the Convention Center where the Esri Developer Summit takes place using [ArcGIS API for Python](https://developers.arcgis.com/python/).

In [1]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

## Required Python Modules
You need to install ArcGIS API for Python, please follow the [guide](https://developers.arcgis.com/python/guide/anaconda/).

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import date, datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban.services import aggregate, query, simulations, top
from geourban.types import GridType, VehicleType

## Connect to ArcGIS Online

In [3]:
gis = GIS()

## Create a map showing the city of Palm Springs

In [4]:
palmsprings_map = gis.map('Palm Springs, United Staates', zoomlevel=13)
palmsprings_map

MapView(layout=Layout(height='400px', width='100%'))

## Connect to geourban services
The host parameter must target the specific host like "geoprotests.p.rapidapi.com". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

Please, check out the [RapidAPI Account Creation and Management Guide](https://docs.rapidapi.com/docs/account-creation-and-settings).

In [5]:
client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')

## List the available simulations

In [6]:
urban_simulations = simulations(client)
urban_simulations

[{'region': 'DEA2D', 'name': 'Aachen, Stadt', 'date': '2023-12-10'},
 {'region': 'USQ27768421',
  'name': 'Allegiant Stadium, Las Vegas',
  'date': '2024-02-11'},
 {'region': 'DE246', 'name': 'Bayreuth, Landkreis', 'date': '2023-12-28'},
 {'region': 'DE300', 'name': 'Berlin', 'date': '2023-08-25'},
 {'region': 'DEA22', 'name': 'Bonn, Kreisfreie Stadt', 'date': '2023-08-24'},
 {'region': 'DEE01', 'name': 'Dessau-Roßlau, Stadt', 'date': '2023-08-24'},
 {'region': 'DED21', 'name': 'Dresden, Stadt', 'date': '2023-12-10'},
 {'region': 'DEA12',
  'name': 'Duisburg, Kreisfreie Stadt',
  'date': '2023-12-22'},
 {'region': 'DE113', 'name': 'Esslingen, Landkreis', 'date': '2023-12-19'},
 {'region': 'ITI14', 'name': 'Firenze', 'date': '2024-01-07'},
 {'region': 'DE712',
  'name': 'Frankfurt am Main, Kreisfreie Stadt',
  'date': '2023-11-21'},
 {'region': 'DE929', 'name': 'Hannover, Region', 'date': '2023-11-09'},
 {'region': 'DE115', 'name': 'Ludwigsburg, Landkreis', 'date': '2023-12-18'},
 {'reg

## Request the top 5 accumulated car traffic grid cells
We request these hotspots for the city of Palm Springs by using the urban region code `USQ115930`, the simulation date `2024-03-07`, the vehicle type `Car`, and the grid type `agent`. The returned GeoJSON features represents the grid cells with the highest car throughput.

In [7]:
palmsprings_region_code = 'USQ115930'
simulation_date = date(2024, 3, 7)
vehicle_type = VehicleType.CAR
grid_type = GridType.AGENT
limit = 5
top_traffic_grid_cells = top(client, palmsprings_region_code, simulation_date, vehicle_type, grid_type, limit=limit)
top_traffic_grid_cells

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-116.53556, 33.8277],
      [-116.53427, 33.8277],
      [-116.53362, 33.82863],
      [-116.53427, 33.82957],
      [-116.53556, 33.82957],
      [-116.53621, 33.82863],
      [-116.53556, 33.8277]]]},
   'properties': {'start_time': '2024-03-07T18:00:00',
    'end_time': '2024-03-07T18:59:59',
    'agent_count': 341}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-116.53751, 33.82863],
      [-116.53621, 33.82863],
      [-116.53556, 33.82957],
      [-116.53621, 33.8305],
      [-116.53751, 33.8305],
      [-116.53816, 33.82957],
      [-116.53751, 33.82863]]]},
   'properties': {'start_time': '2024-03-07T18:00:00',
    'end_time': '2024-03-07T18:59:59',
    'agent_count': 329}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-116.49667, 33.8221],
      [-116.49537, 33.8221],
      [-116.49472, 33.8

## Convert the returned GeoJSON result into a FeatureSet
The FeatureSet offers direct access to a spatially enabled dataframe. We can easily inspect the time frames (`start_time` - `end_time`) and the number of car vehicles `agent_count`.

We see that the highest car throughput is before and after the conference event, e.g. between `07:00:00 - 07:59:59` and `18:00:00 - 18:59:59`.

In [8]:
top_traffic_fset = FeatureSet.from_geojson(top_traffic_grid_cells)
top_traffic_sdf = top_traffic_fset.sdf
top_traffic_sdf

,start_time,end_time,agent_count,OBJECTID,SHAPE
0,2024-03-07T18:00:00,2024-03-07T18:59:59,341,1,"{""rings"": [[[-116.53556, 33.8277], [-116.53427..."
1,2024-03-07T18:00:00,2024-03-07T18:59:59,329,2,"{""rings"": [[[-116.53751, 33.82863], [-116.5362..."
2,2024-03-07T07:00:00,2024-03-07T07:59:59,295,3,"{""rings"": [[[-116.49667, 33.8221], [-116.49537..."
3,2024-03-07T07:00:00,2024-03-07T07:59:59,295,4,"{""rings"": [[[-116.49472, 33.82117], [-116.4934..."
4,2024-03-07T07:00:00,2024-03-07T07:59:59,295,5,"{""rings"": [[[-116.49472, 33.81931], [-116.4934..."
